In [1]:
from urllib.request import urlopen
import json
import plotly.express as px
import geopandas as gpd
from pathlib import Path
import preprocess_data as ppd
import pandas as pd
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash
from dash import html
from dash import dcc
import streamlit as st

data_path = Path("../data")


In [3]:
df = px.data.election()
df.head()

,district,Coderre,Bergeron,Joly,total,winner,result,district_id
0,101-Bois-de-Liesse,2481,1829,3024,7334,Joly,plurality,101
1,102-Cap-Saint-Jacques,2525,1163,2675,6363,Joly,plurality,102
2,11-Sault-au-Récollet,3348,2770,2532,8650,Coderre,plurality,11
3,111-Mile-End,1734,4782,2514,9030,Bergeron,majority,111
4,112-DeLorimier,1770,5933,3044,10747,Bergeron,majority,112


In [3]:
df, adjacent_matrix = ppd.get_df_adj(data_path, 2021)
geodata = gpd.read_file(data_path / "us_counties.geojson")
geodata.id = geodata.id.astype(str).astype(int)
geodata_ohio = geodata[geodata.id.isin(df.fips)]

geo_df = pd.merge(
    left=gpd.GeoDataFrame.from_features(geodata_ohio),
    right=df,
    left_on="id",
    right_on="fips"
).set_index("county_id")
geo_df["colors"] = 0

fig = px.choropleth(geo_df,
                    geojson=geo_df.geometry,
                    hover_name="NAME",
                    locations=geo_df.index,
                    color="colors", template="plotly")
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(height=400, margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()
# geo_df[geo_df.index==4]

In [4]:
geo_df

,geometry,id,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,county,population,fips,adjacent_id,adjacent_names,colors
county_id,,,,,,,,,,,,,,
28,"POLYGON ((-81.00307 41.42593, -81.00319 41.424...",39055,0500000US39055,39,055,Geauga,County,400.164,Geauga County,95408,39055,"[4, 18, 43, 67, 78, 28]","[Ashtabula County, Cuyahoga County, Geauga Cou...",0
32,"POLYGON ((-83.42105 41.16678, -83.42032 40.991...",39063,0500000US39063,39,063,Hancock,County,531.358,Hancock County,75139,39063,"[2, 33, 69, 74, 87, 88, 32]","[Allen County, Hancock County, Hardin County, ...",0
33,"POLYGON ((-83.52023 40.50408, -83.87993 40.538...",39065,0500000US39065,39,065,Hardin,County,470.405,Hardin County,30738,39065,"[2, 6, 32, 46, 51, 80, 88, 33]","[Allen County, Auglaize County, Hancock County...",0
38,"POLYGON ((-82.12620 40.66823, -81.65005 40.668...",39075,0500000US39075,39,075,Holmes,County,422.533,Holmes County,44166,39075,"[3, 16, 42, 76, 79, 85, 38]","[Ashland County, Coshocton County, Holmes Coun...",0
45,"POLYGON ((-82.58181 39.93016, -82.78249 39.939...",39089,0500000US39089,39,089,Licking,County,682.500,Licking County,177454,39089,"[16, 21, 23, 25, 42, 60, 64, 45]","[Coshocton County, Delaware County, Fairfield ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,"POLYGON ((-82.13928 38.86641, -82.18443 38.816...",39053,0500000US39053,39,053,Gallia,County,466.530,Gallia County,29369,39053,"[36, 40, 44, 53, 82, 27]","[Gallia County, Highland County, Jackson Count...",0
34,"POLYGON ((-81.03550 40.16487, -81.22592 40.170...",39067,0500000US39067,39,067,Harrison,County,402.339,Harrison County,14623,39067,"[7, 10, 30, 41, 79, 34]","[Belmont County, Carroll County, Guernsey Coun...",0
37,"POLYGON ((-82.37453 39.65496, -82.37989 39.596...",39073,0500000US39073,39,073,Hocking,County,421.323,Hocking County,28180,39073,"[5, 23, 64, 65, 71, 82, 37]","[Athens County, Fairfield County, Hocking Coun...",0


In [3]:
def get_neighbours(selection, geo_df=geo_df, adjacent_matrix=adjacent_matrix):
    if selection is None:
        return None
    else:
        selection_df = geo_df.loc[adjacent_matrix[selection]]
        selection_df["colors"] = 1
        selection_df.loc[selection, 'colors'] = 3
        return selection_df
hi=get_neighbours(selection=4)
hi["NAME"].values.tolist()

['Geauga', 'Lake', 'Trumbull', 'Ashtabula']

In [4]:
def get_figure(hover):
    fig = px.choropleth(geo_df,
                    geojson=geo_df.geometry,
                    hover_name="NAME",
                    locations=geo_df.index,
                    color="colors")

    if hover!=-1:
        highlights = get_neighbours(hover)
        print_hi = highlights["NAME"].values.tolist()
        print(f"Highlighting for {hover} : {print_hi}\n")

        fig.add_trace(
            px.choropleth(highlights, geojson=highlights.geometry,
                        color="colors",
                        hover_name="NAME",
                        locations=highlights.index).data[0]
        )

    fig.update_geos(fitbounds="locations")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                      uirevision='constant',
                      showlegend=False)
    return fig

In [5]:
hover = -1
app = JupyterDash(__name__)

app.layout = html.Div([    
    dcc.Graph(
        id='choropleth',
        figure=fig,
        responsive=True)
])

#-------------------------------#

@app.callback(
    Output('choropleth', 'figure'),
    [Input('choropleth', 'hoverData')])
def update_figure(hoverData):
    hover = -1  
    if hoverData is not None:
        hover = hoverData['points'][0]['location']
    
    # print(f"Selected: {hover}")
    return get_figure(hover)
    
#-------------------------------#    
# app.run_server(mode='inline', port=8088, debug=True)
app.run_server(debug=True, use_reloader=False, port=8088)

Dash app running on http://127.0.0.1:8088/


Highlighting for 31 : ['Butler', 'Clermont', 'Warren', 'Hamilton']

Highlighting for 31 : ['Butler', 'Clermont', 'Warren', 'Hamilton']

